In [23]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
from datetime import datetime
import pytz
from pymongo import MongoClient
from datetime import timedelta
from urllib.request import Request , urlopen

In [24]:
 client = MongoClient('mongodb://localhost:27017/')

In [25]:
db = client['crime_app']
table=db['crimes']

In [26]:

urls={'miami':'https://citizen.com/api/incident/trending?lowerLatitude=25.672174256489043&lowerLongitude=-80.4344234133622&upperLatitude=25.920839335586436&upperLongitude=-80.01550258663684&fullResponse=true&limit=20', 
      'new york':'https://citizen.com/api/incident/trending?lowerLatitude=40.261686710556404&lowerLongitude=-74.81635630855982&upperLatitude=41.18908240966411&upperLongitude=-73.11564369144105&fullResponse=true&limit=20',
     'austin':'https://citizen.com/api/incident/trending?lowerLatitude=29.738300837278217&lowerLongitude=-98.60285830855923&upperLatitude=30.795222582623737&upperLongitude=-96.90214569144081&fullResponse=true&limit=20',
     'baltimore':'https://citizen.com/api/incident/trending?lowerLatitude=39.21772220334134&lowerLongitude=-76.75488050418187&upperLatitude=39.38018353927404&upperLongitude=-76.46311949581934&fullResponse=true&limit=20',
     'cleveland':'https://citizen.com/api/incident/trending?lowerLatitude=41.39810311215828&lowerLongitude=-81.8263119167396&upperLatitude=41.55539424005582&upperLongitude=-81.53455090837701&fullResponse=true&limit=20',
     'cincinnati':'https://citizen.com/api/incident/trending?lowerLatitude=38.91357469384428&lowerLongitude=-84.9335164270882&upperLatitude=39.42743853297762&upperLongitude=-84.01236018664895&fullResponse=true&limit=20',
     'columbus': 'https://citizen.com/api/incident/trending?lowerLatitude=39.88761983506768&lowerLongitude=-83.14172530418085&upperLatitude=40.048517289309416&upperLongitude=-82.84996429581832&fullResponse=true&limit=20',
     'charlotte':'https://citizen.com/api/incident/trending?lowerLatitude=35.32337228624063&lowerLongitude=-84.84960066238546&upperLatitude=38.93552269689269&upperLongitude=-78.55095087814328&fullResponse=true&limit=20',
     'chicago':'https://citizen.com/api/incident/trending?lowerLatitude=41.773313054724724&lowerLongitude=-87.804520504182&upperLatitude=41.92969137030829&upperLongitude=-87.51275949581947&fullResponse=true&limit=20',
     'detroit':'https://citizen.com/api/incident/trending?lowerLatitude=39.448137919239514&lowerLongitude=-84.601770178636&upperLatitude=44.164173923741146&upperLongitude=-75.80265761144233&fullResponse=true&limit=20',
     'houston':'https://citizen.com/api/incident/trending?lowerLatitude=29.620057015864077&lowerLongitude=-95.58294673207138&upperLatitude=29.879375284648717&upperLongitude=-95.16785126792848&fullResponse=true&limit=20',
     'indianapolis':'https://citizen.com/api/incident/trending?lowerLatitude=39.373519878846224&lowerLongitude=-86.87262162459285&upperLatitude=40.162111876327884&upperLongitude=-85.44678237540683&fullResponse=true&limit=20',
    'los angeles':'https://citizen.com/api/incident/trending?lowerLatitude=33.64299720998807&lowerLongitude=-119.05491962459303&upperLatitude=34.49287152662065&upperLongitude=-117.6290803754074&fullResponse=true&limit=20',
     'minneapolis':'https://citizen.com/api/incident/trending?lowerLatitude=44.60903023809183&lowerLongitude=-93.97714262459374&upperLatitude=45.334843332235636&upperLongitude=-92.55130337540777&fullResponse=true&limit=20',
     'philadelphia':'https://citizen.com/api/incident/trending?lowerLatitude=39.837420868282095&lowerLongitude=-75.3735477320709&upperLatitude=40.06638751389556&upperLongitude=-74.95845226792846&fullResponse=true&limit=20',
     'phoenix':'https://citizen.com/api/incident/trending?lowerLatitude=33.39240309192556&lowerLongitude=-112.22388050418095&upperLatitude=33.567508423126355&upperLongitude=-111.93211949581814&fullResponse=true&limit=20',
     'san francisco':'https://citizen.com/api/incident/trending?lowerLatitude=37.38890933552479&lowerLongitude=-122.59020790312877&upperLatitude=37.79054881168709&upperLongitude=-121.88579209687083&fullResponse=true&limit=20',
     'stockton':'https://citizen.com/api/incident/trending?lowerLatitude=37.90651339148734&lowerLongitude=-121.42075593728612&upperLatitude=38.04525504465582&upperLongitude=-121.1761500627145&fullResponse=true&limit=20',
     'tucson':'https://citizen.com/api/incident/trending?lowerLatitude=31.533091897097748&lowerLongitude=-112.0677493154426&upperLatitude=32.881907338344476&upperLongitude=-109.85225068455782&fullResponse=true&limit=20'}

In [27]:
editable_list=db['list_of_resources']
editable_list=editable_list.find_one({'Network':'Citizen App'})
list_of_cities=editable_list['Locations']
#to_find=['new york','los angeles','san francisco']

In [28]:
list_of_cities

['new york', 'los angeles', 'san francisco']

In [29]:
links={}

In [30]:
for item in list_of_cities:
    keys=urls.keys()
    for key in keys:
        if item.lower() in key:
            links[item]=urls[key]

In [31]:
links

{'new york': 'https://citizen.com/api/incident/trending?lowerLatitude=40.261686710556404&lowerLongitude=-74.81635630855982&upperLatitude=41.18908240966411&upperLongitude=-73.11564369144105&fullResponse=true&limit=20',
 'los angeles': 'https://citizen.com/api/incident/trending?lowerLatitude=33.64299720998807&lowerLongitude=-119.05491962459303&upperLatitude=34.49287152662065&upperLongitude=-117.6290803754074&fullResponse=true&limit=20',
 'san francisco': 'https://citizen.com/api/incident/trending?lowerLatitude=37.38890933552479&lowerLongitude=-122.59020790312877&upperLatitude=37.79054881168709&upperLongitude=-121.88579209687083&fullResponse=true&limit=20'}

In [32]:
datetime_now=datetime.now(tz=pytz.timezone('EET'))
print(datetime_now)
delta=timedelta(days=1)
datetime_now=datetime_now-delta
print(datetime_now)
cursor=table.find({'Network':'Citizen App', 'Location': { '$in':list_of_cities },'Date':{'$gte':datetime.strptime(datetime_now.strftime('%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S')}})

2020-09-28 11:58:13.101875+03:00
2020-09-27 11:58:13.101875+03:00


In [33]:
old_list=[]
for row in cursor:
    
    old_list.append(row['Key'])   
    #print(row)

In [34]:
old_list

[]

In [35]:
list_of_keys=[]
list_of_address=[]
list_of_latitude=[]
list_of_longitude=[]
list_of_types=[]
list_of_description=[]
list_of_datetime=[]
list_of_map_image_url=[]

In [36]:
j=1
for link in links:
    print(link)
    data=requests.get(links[link])
    data=data.json()
    news_items=data['results']
    try:
        for news_item in news_items:
            news_key=news_item['key']
            news_key=str(news_key)
            if news_key in old_list:
                print('Found in old')
                j=1
                raise Exception
            list_of_keys.append(news_key)
        #print(news_key)
            address=news_item['address']
            list_of_address.append(address)
            latitude=news_item['latitude']
            list_of_latitude.append(latitude)
            longitude=news_item['longitude']
            list_of_longitude.append(longitude)
            try:
                category=news_item['categories'][0]
            except:
                category='Unspecified'
            list_of_types.append(category)
            description=''
            try:
                description=description+news_item['raw']
            except:
                print()
            try:
                latest_update=news_item['updates']
                latest_update=list(latest_update.items())[0][1]['text']
                description=description+latest_update
            except Exception:
                print()
            list_of_description.append(description)
            try:
                latest_update_time=news_item['updates']
                latest_update_time=list(latest_update_time.items())
                integers=[]
                for key,value in latest_update_time:
                    integers.append(int(value['ts']))
                stamp=min(integers)
                stamp=int(stamp/1000)
                tz = pytz.timezone('EET')
                dt_object = datetime.fromtimestamp(stamp , tz=tz)
                date_corrected=dt_object.strftime("%Y-%m-%d %H:%M:%S")
            #print(date_corrected)
                list_of_datetime.append(date_corrected)
            except:
                list_of_datetime.append('')
            videolinks=[]
            if 'liveStreamers' in news_item:
                videos=news_item['liveStreamers']
                for video in videos:
                    video_id=videos[video]['videoStreamId']
                    video_link='https://citizen.com/'+news_key+'?media='+video_id
                    videolinks.append(video_link)
                #print(video_link)
            image_url=news_item['shareMap']  
            list_of_map_image_url.append(image_url)
            document={'Key':news_key,'Longitude':longitude, 'Latitude':latitude, 'Location':link,'Text':description,'Category':category, 'Address':address
                        , 'Image_url':image_url , 'Video_Sources':videolinks, 'Date':dt_object, 'Network': 'Citizen App'}
            print(j)
            j=j+1
            table.insert_one(document)
    except:
        continue

new york
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
los angeles
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
san francisco
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [156]:
https://citizen.com/-MHZ3r4TCaxZFbX4qKqw?media=a8f655d4-fea7-4af7-bfb1-de1f3c0fe722

SyntaxError: invalid syntax (<ipython-input-156-203a2ad3eda5>, line 1)

In [16]:
df=pd.DataFrame({'Key':list_of_keys,'Type':list_of_types,'Description':list_of_description,'Address':list_of_address,'DateTime':list_of_datetime,'Latitude':list_of_latitude,'Longitude':list_of_longitude,'Map':list_of_map_image_url})

In [17]:
df.to_csv('citizenapp.csv', index=False)
df.head()

,Key,Type,Description,Address,DateTime,Latitude,Longitude,Map
0,-MI2DQL7EqYuJvR344tI,Fire,"Dumpster Fire at 10 Hudson Yards, New York, NY...","10 Hudson Yards, New York, NY 10001, USA",2020-09-25 06:06:12,40.753687,-74.001916,https://storage.googleapis.com/citizen-notif-a...
1,-MI28QMM2ZtMYqTz3TXU,Weapon,"Deli Robbed at Gunpoint at 2336 Bedford Ave, B...","2336 Bedford Ave, Brooklyn, NY 11226, USA",2020-09-25 05:44:22,40.646747,-73.955873,https://storage.googleapis.com/citizen-notif-a...
2,-MI26ZDmGRPGrvHNsoMJ,Protest,Protesters March to Williamsburg Bridge at Chr...,"Chrystie Street & Grand Street, Manhattan, New...",2020-09-25 05:36:13,40.718166,-73.993847,https://storage.googleapis.com/citizen-notif-a...
3,-MI22GlThtyFlEe_Vi8b,Fire,"Apartment Fire at 4530 Broadway, Manhattan, Ne...","4530 Broadway, Manhattan, New York, NY, USA",2020-09-25 05:17:29,40.858711,-73.931332,https://storage.googleapis.com/citizen-notif-a...
4,-MI1wrDx3WlgAhjMf221,Weapon,"Man Stabbed at Park Ave & E 116th St, New York...","Park Ave & E 116th St, New York, NY 10029, USA",2020-09-25 04:49:28,40.799247,-73.943099,https://storage.googleapis.com/citizen-notif-a...


In [18]:
df.shape

(60, 8)